In [166]:
def parenthese_match(string):
    """take from string until matching close parenthese found.
    string should start with the ( that is to be to matched."""
    
    open_parentheses = 0
    close_parentheses = 0
    
    index = 0
    
    for char in string:
        if char == "(":
            open_parentheses += 1
            
        elif char == ")":
            close_parentheses += 1
            
            if open_parentheses == close_parentheses:
                index = string.index(char)
                index += close_parentheses
                break
                
    if index == 0:
        return "Parse error. Trailing close parentheses not found."
    
    else:
        return string[0:index]

In [167]:
parenthese_match("(CI(KI))(CIK)")

'(CI(KI))'

In [168]:
parenthese_match("(KI)(CIK)")

'(KI)'

In [169]:
parenthese_match("(KI(WI(BI)))(CIK)")

'(KI(WI(BI)))'

Example expression `S(KS)Kxyz` and example list `["S",["K","S"],"K","x","y","z"]`

Another example expression `S(CI(KI))(CIK)` and example list `["S",["C","I",["K","I"]],["C","I","K"]]`

In [263]:
def encoding(data):
    """CL string to list OR CL list to string"""
    
    if type(data) == str:
        d = data 
        _list = []
        
        while len(d) > 0:
            if d[0] == "(":
                p = parenthese_match(d) 
                _list.append(encoding(p[1:-1])) 
                d = d[len(p):]
                
            elif d[0] == ")":
                d = d[1:]
                                
            else:
                _list.append(d[0])
                d = d[1:]
        
        if len(_list) == 1 and type(_list[0]) == list:
            return _list[0]
        
        else:
            return _list
        
    elif type(data) == list:
        string = ""
        
        for i in data:
            if type(i) == str:
                string += i
            else:
                string += "("
                string += encoding(i)
                string += ")"
        
        return string 
        
    else:
        return f"Type Error. Got: {type(data)}, Expected 'str' or 'list'"

In [264]:
encoding(["S",["K","S"],"K","x","y","z"])

'S(KS)Kxyz'

In [265]:
encoding(["S",["C","I",["K","I"]],["C","I","K"]])

'S(CI(KI))(CIK)'

In [266]:
encoding((1,2))

"Type Error. Got: <class 'tuple'>, Expected 'str' or 'list'"

In [267]:
encoding("S(KS)Kxyz")

['S', ['K', 'S'], 'K', 'x', 'y', 'z']

In [268]:
encoding("S(KS)Kxyz") == ["S",["K","S"],"K","x","y","z"]

True

In [269]:
encoding("S(CI(KI))(CIK)")

['S', ['C', 'I', ['K', 'I']], ['C', 'I', 'K']]

In [270]:
encoding("S(CI(KI))(CIK)") == ["S",["C","I",["K","I"]],["C","I","K"]]

True

-- --

In [322]:
def I_combinator(string):
    """Ix = x"""
    return string

def K_combinator(string):
    """Kxy = x"""
    return encoding(string)[0]

def B_combinator(string):
    """Bxyz = x(yz)"""
    l = encoding(string)
    return encoding([l[0],[l[1],l[2]]])

def C_combinator(string):
    """Cxyz = xzy"""
    l = encoding(string)
    return encoding([l[0],l[2],l[1]])
    
def W_combinator(string):
    """Wxy = xyy"""
    l = encoding(string)
    return encoding([l[0],l[1],l[1]])

def S_combinator(string):
    """Sxyz = xz(yz)"""
    l = encoding(string)
    return encoding([l[0],l[2],[l[1],l[2]]])

def Y_combinator(string):
    """Yx = B(WI)(BWB)x = ?"""
    l = encoding(string)
    return B_combinator("(WI)(BWB)" + string)

In [323]:
I_combinator("x")

'x'

In [324]:
K_combinator("xy")

'x'

In [325]:
B_combinator("xyz")

'x(yz)'

In [326]:
C_combinator("xyz")

'xzy'

In [327]:
W_combinator("xy")

'xyy'

In [328]:
S_combinator("xyz")

'xz(yz)'

In [329]:
Y_combinator("x")

'(WI)((BWB)x)'

-- --

In [330]:
encoding("(ABC)")

['A', 'B', 'C']

In [331]:
m = {"S": S_combinator}

In [332]:
m["S"]("xyz")

'xz(yz)'

In [367]:
def evaluate(string):
    cl_list = encoding(string)
    
    while type(cl_list[0]) == list: # strip leading parentheses
        cl_list = [i for i in cl_list[0]] + cl_list[1:]
    
    # "X": (function, arg_num)
    _map = {"I": (I_combinator,1), "K": (K_combinator,2), "B": (B_combinator,3), "C": (C_combinator,3), 
            "W": (W_combinator,2), "S": (S_combinator,3), "Y": (Y_combinator,1)}
    
    combinator_return = _map[cl_list[0]][0](encoding(cl_list[1:(_map[cl_list[0]][1]+1)]))
    tail = encoding(cl_list[(_map[cl_list[0]][1]+1):])
    
    return combinator_return + tail

In [368]:
evaluate("S(KS)Kxyz")

'(KS)x(Kx)yz'

In [369]:
evaluate("(KS)x(Kx)yz")

'S(Kx)yz'

In [370]:
evaluate("S(Kx)yz")

'(Kx)z(yz)'

In [371]:
evaluate("(Kx)z(yz)")

'x(yz)'

-- --

In [ ]:
def recur(string,steps=[],print_steps=False,write_steps=False):
    steps.append(string)
    
    while True:
        try:
            ns = evaluate(string)
            steps.append(ns)
            recur(ns,steps)
        except:
            break
    
    if print_steps:
        for i in steps:
            print(i)
            
    if write_steps:
        file = input("filename: ")
        # IO
        
    return steps[-1]

# print(f"{string} = {recur(string)}")